In [5]:
from datasets import Dataset
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings

embeddings =
llm =

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[faithfulness, answer_correctness], llm=llm, embeddings=embeddings)
score.to_pandas()

Exception in thread Thread-25:
Traceback (most recent call last):
  File "c:\Users\luisb\.pyenv\pyenv-win\versions\3.12.2\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\luisb\Documents\GitHub\documentQuestionAnswering\.venv\Lib\site-packages\ragas\executor.py", line 87, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luisb\.pyenv\pyenv-win\versions\3.12.2\Lib\asyncio\base_events.py", line 661, in run_until_complete
    self._check_running()
  File "c:\Users\luisb\.pyenv\pyenv-win\versions\3.12.2\Lib\asyncio\base_events.py", line 620, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running
c:\Users\luisb\.pyenv\pyenv-win\versions\3.12.2\Lib\threading.py:1075: RuntimeWarning: coroutine 'Runner._aresults' was never awaited
  self._invoke_excepthook(self)


ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

c:\Users\luisb\Documents\GitHub\documentQuestionAnswering\.venv\Lib\site-packages\ipykernel\iostream.py:123: RuntimeWarning: coroutine 'as_completed.<locals>.sema_coro' was never awaited
  await self._event_pipe_gc()
